In [ ]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch

In [ ]:
role = sagemaker.get_execution_role()
git_config=git_config = {
    "repo": "https://github.com/CrystalGazers/TAED2_CrystalGazers",
    "branch": "training-scripts",
},

In [ ]:
pytorch_estimator = PyTorch('SM_train_model.py',
                            instance_type='ml.t3.medium',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            role=role,
                            git_config=git_config,
                            hyperparameters = {'epochs': 4, 'batch-size': 2048, 'embedding_dim': 256})
                            
pytorch_estimator.fit({'train': 's3://my-data-bucket/path/to/my/training/data',
                       'test': 's3://my-data-bucket/path/to/my/test/data'})